# Content

In this notebook we provide a demo on how to use our proposed algorithm for node classification in weighted graphs. We will also provide the elementary tools to recreate the outcomes of Figures 6, 7, 8.

In [59]:
# upload the source files

include("src/basic_functions.jl")
include("src/NBNC.jl")
include("src/clustering_algorithms.jl")

using Plots, LaTeXStrings ## these packages are used just for the plots
using DelimitedFiles ## this is to upload the GAN features



gr(size=(140,100), xtickfontsize = 4, ytickfontsize = 4, linewidth = 0.5, legendfontsize = 2, markersize = 2)

;

## Clustering on synthetic data

With the following code we show the performance of label inference on synthetic data, according to the following steps:

* Generate the graph $\mathcal{G}(\mathcal{V},\mathcal{E})$. One can simply generate a Erdos-Renyi graph or its degree-corrected version, to obtain an arbitrary degree distribution.
* Assign the weights to each edge. The weights are assigned keeping under consideration the class labels that have to be estimated. We here propose two weights assignments: the Gaussian weight and the binary weight.
* Run the algorithms based on the ``Nishimori Bethe-Hessian``, the ``spin-glass Bethe-Hessian``, the ``na\"ive mean field`` method and finally the ``weighted Laplacian`` matrix. All algorithms output the estimated labels and the vector ``X`` from which the labels have been estimated.
* We finally show the performance of reconstruction in terms of overlap for all the algorithms.

In [75]:
@time begin

n = 3000 # number of nodes
c = 8. # average degree
    
ℓ = ones(n) # label vector
ℓ[1:Int(n/2)] .= -1

""" Now we build the adjacency matrix. If, instead of using a Erdos Renyi graph, one wants to create a heterogeneous
    degree distribution like in the one used in Figure 7, instead of using the function ```adjacency_matrix_ER```, 
    use the function ```adjacency_matrix_DCER``` (commented below). Note that the implementation of the function
    ```adjacency_matrix_DCER``` is adapted only to sparse graphs.
    
"""

A, edge_list = adjacency_matrix_ER(c,n) # generate the adjacency matrix of the Erdos-Renyi graph
    
# To obtain an arbitrary degree distribution uncomment the following lines of code
    
# θ = rand(Uniform(3,10),n).^4 # this vector θ generates a power law degree distribution
# θ = θ/mean(θ)
# A, edge_list =  adjacency_matrix_DCER(c,θ)

    
""" Now we initialize the parameters for the simulation """
    
μ = 1. # mean of the Gaussian
ν = 2. # standard deviation of the Gaussian
p = 0.7

verbose = 2
    

J_edge_list = zeros(length(edge_list[:,1])) # assign the weights to each edge according to Equation 27

    
""" With this bit of code we get a weighted graph. In this case β_N = μ/ν^2 """
    
for k=1:length(J_edge_list)
    a = edge_list[k,1]
    b = edge_list[k,2]
    J_edge_list[k] = rand(Normal(μ*ℓ[a]*ℓ[b],ν))
end
    
""" With this bit of code we obtain a signed graph. In this case β_N = atanh(2p-1) """

# for k=1:length(J_edge_list)
#     a = edge_list[k,1]
#     b = edge_list[k,2]
#     rn = rand(Uniform(0,1))
#     if rn < p
#         J_edge_list[k] = ℓ[a]*ℓ[b]
#      else
#         J_edge_list[k] = -ℓ[a]*ℓ[b]
#     end
# end
    
    
    
# Run the four clustering algorithms discussed in the article

X, estimated_ℓ = clustering_BH_Nishimori(edge_list, J_edge_list, n, verbose = verbose)
X_LAP, estimated_ℓ_LAP = clustering_signed_Lap(edge_list, J_edge_list, n, verbose = verbose)
X_MF, estimated_ℓ_MF = clustering_MF(edge_list, J_edge_list, n, verbose = verbose)
X_SG, estimated_ℓ_SG = clustering_BH_SG(edge_list, J_edge_list, n)

Overlap = abs(2*(sum(estimated_ℓ .== ℓ)/n - 0.5))
OverlapLAP = abs(2*(sum(estimated_ℓ_LAP .== ℓ)/n - 0.5))
OverlapMF = abs(2*(sum(estimated_ℓ_MF .== ℓ)/n - 0.5))
OverlapSG = abs(2*(sum(estimated_ℓ_SG .== ℓ)/n - 0.5))

    
end


print("\nThe overlaps obtained are:\n",
 "\n BH Nishimori = ", Overlap,
"\n BH spin glass = ", OverlapSG,
"\n Mean field = ", OverlapMF,
"\n Laplacian = ", OverlapLAP)

o The value of β_SG is 0.17. Computing β_N

Iteration # 1: 
The current estimate of β_N is 0.2383169502138014
The smallest eigenvalue is -0.16378928067361181

Iteration # 2: 
The current estimate of β_N is 0.2489633329220985
The smallest eigenvalue is -0.04060721587161855

Iteration # 3: 
The current estimate of β_N is 0.24918818696887746
The smallest eigenvalue is -0.0008493867954859237

Iteration # 4: 
The current estimate of β_N is 0.24918818696887746
The smallest eigenvalue is -3.873945201697472e-7

o The value of β_N is 0.25
o Running kmeans
o Done!
o Running kmeans
o Done!
o Running kmeans
o Done!
o The value of β_SG is 0.17
o Running kmeans
o Done!
  0.626797 seconds (174.49 k allocations: 774.657 MiB, 3.06% gc time)

The overlaps obtained are:

 BH Nishimori = 0.6106666666666667
 BH spin glass = 0.5920000000000001
 Mean field = 0.002666666666666706
 Laplacian = 0.0013333333333334085

## Clustering on real data

Here we show how to implement our algorithm for data clustering. The steps are the following:

* Upload the raw data: the matrix $Y \in \mathbb{R}^{n \times p}$ contains the data of the features extracted from the GAN images. We attached to files: ``features.dat``, used to produce Figure 8 with 40k images and ``features_small.dat`` with 6k images for a faster implementation. In both datasets, the first half of the images belong to one class and the second half to the other.
* The mask $S$ is applied to the signal, obtaining the input matrix $X$.
* We then build the weighted graph, first building a Erdos-Renyi, then assigning to each edge $(ij)$ the weight $\frac{1}{p}x_i^Tx_j$
* We then shift the average of the weights to zero
* Finally we run the different algorithms and compare the performance in terms of overlap.


In [79]:
Y = readdlm("data/features.dat") ## upload the data

# Y = readdlm("data/features_small.dat") ## upload the data

;

In [81]:
@time begin
    
n = length(Y[:,1]) # number of nodes
p = length(Y[1,:]) # number of features
    

c = 10. # average degree
ϵ = 2*10^(-5) # precision error
verbose = 2
       
    
κ = sqrt(p/p)
S = rand(Binomial(1,κ), (n,p))
X = Y.*S # apply the mask to the signal

ℓ = ones(n)
ℓ[1:Int(n/2)] .= -1
        
A, edge_list = adjacency_matrix_ER(c,n) # generate the adjacency matrix of the Erdos-Renyi graph

J_edge_list = zeros(length(edge_list[:,1]))

for k=1:length(J_edge_list)
    a = edge_list[k,1]
    b = edge_list[k,2]
    J_edge_list[k] = X[a,:]'*X[b,:]/p # covariance matrix
end

J_edge_list = J_edge_list .- mean(J_edge_list) # shifht the non-zero entries
# J_edge_list = sign.(J_edge_list) # take a signed representation of the graph

# Run the four clustering algorithms discussed in the article

X, estimated_ℓ = clustering_BH_Nishimori(edge_list, J_edge_list, n, verbose = verbose)
X_LAP, estimated_ℓ_LAP = clustering_signed_Lap(edge_list, J_edge_list, n, verbose = verbose)
X_MF, estimated_ℓ_MF = clustering_MF(edge_list, J_edge_list, n, verbose = verbose)
X_SG, estimated_ℓ_SG = clustering_BH_SG(edge_list, J_edge_list, n)

Overlap = abs(2*(sum(estimated_ℓ .== ℓ)/n - 0.5))
OverlapLAP = abs(2*(sum(estimated_ℓ_LAP .== ℓ)/n - 0.5))
OverlapMF = abs(2*(sum(estimated_ℓ_MF .== ℓ)/n - 0.5))
OverlapSG = abs(2*(sum(estimated_ℓ_SG .== ℓ)/n - 0.5))

    
end


print("\nThe overlaps obtained are:\n",
 "\n BH Nishimori = ", Overlap,
"\n BH spin glass = ", OverlapSG,
"\n Mean field = ", OverlapMF,
"\n Laplacian = ", OverlapLAP)

o The value of β_SG is 0.31. Computing β_N

The signed representation of J is adopted. If you want to use the weighted one, increase the value of `is_signed_th`. The algorithm might have a sensible slow down

Iteration # 1: 
The current estimate of β_N is 1.250011606354928
The smallest eigenvalue is -11.942699185708316

Iteration # 2: 
The current estimate of β_N is 1.4288754042783092
The smallest eigenvalue is -0.4002598380955845

Iteration # 3: 
The current estimate of β_N is 1.430612779368753
The smallest eigenvalue is -0.003117815806615139

Iteration # 4: 
The current estimate of β_N is 1.430612779368753
The smallest eigenvalue is -2.2613805496545056e-7

o The value of β_N is 1.43
o Running kmeans
o Done!
o Running kmeans
o Done!
o Running kmeans
o Done!
o The value of β_SG is 0.31
o Running kmeans
o Done!
  7.389867 seconds (3.39 M allocations: 2.990 GiB, 1.09% gc time)

The overlaps obtained are:

 BH Nishimori = 0.9728000000000001
 BH spin glass = 0.97125
 Mean field = 0.9702500